In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon-Vine-Analysis").getOrCreate()

### NOTE: The Amazon Gift Card dataset I used in the previous portion of the Challenge (https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz) does not appear to utilize any paid Amazon Vine reviewers. That makes the calculations at the end of this deliverable useless since the paid 5-star rating will be zero, and leave us with nothing to compare. I chose another dataset for this portion of the challenge.

In [34]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [35]:
# Filter the DataFrame for total_votes above or equal to 20
twentydf = df.filter(df.total_votes >= 20)
twentydf.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [36]:
# Filter the DataFrame for helpful_votes ratio is 50% or above
helpfuldf = twentydf.filter((twentydf.helpful_votes / twentydf.total_votes) >= 0.5)
helpfuldf.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [37]:
# Paid Vine DataFrame
paid_df = helpfuldf.filter(helpfuldf.vine == "Y")
paid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44759540|R3A71VR1JZD8WF|B00JGWN8O8|     378897034|Petcube Interacti...|    Pet Products|          2|           27|         30|   Y|                N|Great idea, not s...|I do not hate thi...| 2015-08-29|
|         US|   37388532|R16OMUJIGI18JZ|B00IP05CUA|     833937853|Wellness Kittles ...|    Pet Products|          5|    

In [38]:
# Unpaid Vine DataFrame
unpaid_df = helpfuldf.filter(helpfuldf.vine == "N")
unpaid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [39]:
# Total number of reviews overall
df.count()

2643619

In [40]:
# Count of paid reviews
total_paid_reviews = paid_df.count()
total_paid_reviews

170

In [41]:
# Count of unpaid reviews
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

37840

In [42]:
# Count of all reviews, paid and unpaid, that met our criteria of 20+ votes with 50% or more of them being marked as helpful.
total_reviews = total_paid_reviews + total_unpaid_reviews
total_reviews

38010

In [43]:
# Count of 5-star paid reviews
paid_fiver_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_fiver_reviews

65

In [44]:
# Count of 5-star unpaid reviews
unpaid_fiver_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_fiver_reviews

20612

In [45]:
# Percentage of paid reviews with 5 stars
paid_fiver_percent = (paid_fiver_reviews / total_paid_reviews) * 100
paid_fiver_percent

38.23529411764706

In [46]:
# Percentage of unpaid reviews with 5 stars
unpaid_fiver_percent = (unpaid_fiver_reviews / total_unpaid_reviews) * 100
unpaid_fiver_percent

54.471458773784356